<a href="https://colab.research.google.com/github/MathMachado/Python_RFB/blob/master/Fake%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PENDÊNCIAS DESTA SESSÃO

https://medium.com/district-data-labs/a-practical-guide-to-anonymizing-datasets-with-python-faker-ecf15114c9be



# Generating Realist Fake Data

Como a librarie 'faker' não estava instalada, então o comando abaixo foi necessário para instalar a librarie 'faker' juntamente com todas as dependências necessárias para funcionar corretamente.

In [0]:
!pip install Faker

Chama a library...

In [0]:
from faker import Faker
import datetime
import random
import pandas as pd

Instancia...

In [0]:
fake = Faker('pt_BR')

Gerando vários nomes fake...

In [0]:
for _ in range(10):
    print(fake.name())

Bom, a partir de agora vamos definir um seed para garantir que todos gerem os mesmos dados e, claro, garantir os mesmos resultados.

In [0]:
fake.seed(20111974)
fake.seed_instance(20111974)
random.seed(20111974)

In [0]:
# Fonte: https://www.tonic.ai/post/how-to-generate-simple-test-data-with-faker/
def first_name_and_gender():
    g = 'M' if random.randint(0,1) == 0 else 'F'
    n = fake.first_name_male() if g=='M' else fake.first_name_female()
    return {'gender':g,'first_name':n}

In [0]:
# Fonte: https://www.tonic.ai/post/how-to-generate-simple-test-data-with-faker/
def birth_and_start_date():
    sd = fake.date_between(start_date="-20y", end_date="now")
    delta = datetime.timedelta(days=365*random.randint(18,40))
    bd = sd-delta

    return {'birth_date':bd, 'start_date': sd}

In [0]:
# Fonte: https://www.tonic.ai/post/how-to-generate-simple-test-data-with-faker/
def title_office_org():
    #generate a map of real office to fake office
    offices = ['New York','Austin','Seattle','Chicago']
    #codify the hierarchical structure
    allowed_orgs_per_office = {'New York':['Sales'],'Austin':['Devops','Platform','Product','Internal Tools'],'Chicago':['Devops'], 'Seattle':['Internal Tools','Product']}
    allowed_titles_per_org = {
        'Devops':['Engineer','Senior Engineer','Manager'],
        'Sales':['Associate'],
        'Platform':['Engineer'],
        'Product':['Manager','VP'],
        'Internal Tools':['Engineer','Senior Engineer','VP','Manager']
    }

    office = random.choice(offices)
    org = random.choice(allowed_orgs_per_office[office])
    title = random.choice(allowed_titles_per_org[org])
    return {'ofice':office, 'title':title,'org': org}

In [0]:
# Fonte: https://www.tonic.ai/post/how-to-generate-simple-test-data-with-faker/
def salary_and_bonus():
    salary = round(random.randint(90000,120000)/1000)*1000
    bonus_ratio = random.uniform(0.15,0.2)
    bonus = round(salary*bonus_ratio/500)*500
    return {'salary':salary,'bonus':bonus}

In [0]:
# Fonte: https://www.tonic.ai/post/how-to-generate-simple-test-data-with-faker/
def title_office_org_salary_bonus():
    position = title_office_org()
    title_and_salary_range = {'Engineer':[90,120],'Senior Engineer':[110,140],'Manager':[130,150],'Associate':[60,80],'VP':[150,250]}
    salary_range = title_and_salary_range[position['title']]

    salary = round(random.randint(1000*salary_range[0],1000*salary_range[1])/1000)*1000
    bonus_ratio = random.uniform(0.15,0.2)
    bonus = round(salary*bonus_ratio/500)*500
    position.update({'salary':salary,'bonus':bonus})
    return position

In [0]:
d = dict()
d['first_name_and_gender'] = first_name_and_gender
d['last_name'] = lambda: {'last_name':fake.last_name()}
d['personal_email'] = lambda: {'email':fake.email()}
d['ssn'] = lambda: {'ssn':fake.ssn()}
d['birth_and_start_date'] = birth_and_start_date
d['title_office_org_salary_bonus'] = title_office_org_salary_bonus
d['accrued_holidays'] = lambda: {'accrued_holiday':random.randint(0,20)}

Definindo um Dataframe vazio com o nome das colunas que serão geradas de forma fake.

In [0]:
My_df= pd.DataFrame(columns=['Sex', 'First_Name', 'Second_Name', 'email', 'ssn', 'Birth_date', 'start_date', 'City', 'Job', 'Department', 'Anual_Salary', 'Bonus', 'accrued_holidays'])
My_df

In [0]:
for _ in range(1):
    deep_list = [list(d[k]().values()) for k in d.keys()]
    row = [item for sublist in deep_list for item in sublist]
    My_df.append(row, ignore_index=True)
    print(row)